In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Load the data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_data['text'], train_data['label'], test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train).toarray()
X_val_vec = vectorizer.transform(X_val).toarray()
X_test_vec = vectorizer.transform(test_data['text']).toarray()

# Define the neural network model
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=X_train_vec.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
model.fit(X_train_vec, pd.get_dummies(y_train), epochs=50, batch_size=64, validation_data=(X_val_vec, pd.get_dummies(y_val)), callbacks=[early_stopping])

# Make predictions on the test data
y_pred_prob = model.predict(X_test_vec)
y_pred = y_pred_prob.argmax(axis=1)

# Save the predictions to a CSV file
submission_df = pd.DataFrame({'id': test_data['id'], 'label': y_pred})
submission_df.to_csv('submission.csv', index=False)


Epoch 1/50
520/593 [=========================>....] - ETA: 1s - loss: 0.6449 - accuracy: 0.7907

KeyboardInterrupt: 